In [1]:
import time
import numpy as np
from pandas import DataFrame
import matplotlib.pyplot as plt
import dgl
import networkx as nx
import torch
import torch.nn as nn

import utils
from gsp_utils.baselines_archs import GCNN_2L
from gsp_utils.baselines_models import NodeClassModel, GF_NodeClassModel
from gsp_utils.data import normalize_gso
from src.arch import GFGCN, GFGCNLayer, GFGCN_noh_Layer, GFGCN_Spows

SEED = 15
# PATH = 'results/diff_filters/'
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
print(device)

torch.manual_seed(SEED)

cuda:1


In [2]:
# Dataset must be from DGL
dataset_name = 'CornellDataset'

A, feat, labels, n_class, masks = utils.get_data_dgl(dataset_name, dev=device,
                                                     verb=True)
N = A.shape[0]

Dataset: CornellDataset
Number of nodes: 183
Number of features: 1703
Shape of signals: torch.Size([183, 1703])
Number of classes: 5
Norm of A: 17.262676239013672
Max value of A: 1.0
Proportion of validation data: 0.32
Proportion of test data: 0.20
Node homophily: 0.11
Edge homophily: 0.13


# Without normalizing the GSO

In [4]:
# BEST PARAMETERS
## Reaining params
N_RUNS = 25
N_EPOCHS = 200  # 500
LR = .05
WD = .01
DROPOUT = 0

# BEST PARAMETERS
## Architecture params
N_LAYERS = 2
K = 3
HID_DIM = 50

## Model params
h0 = .1  # 1
NORM = False

IN_DIM = feat.shape[1]
OUT_DIM = n_class


ACT = nn.ReLU()
LAST_ACT = nn.Softmax(dim=1)
LOSS_FN = nn.CrossEntropyLoss()

## Training paramerters

In [4]:
EXPS = [{'epochs': 200, 'lr': .1, 'wd': 5e-4},
        {'epochs': 200, 'lr': .05, 'wd': 5e-4},
        {'epochs': 200, 'lr': .01, 'wd': 5e-4},
        {'epochs': 200, 'lr': .005, 'wd': 5e-4},
        
        {'epochs': 200, 'lr': .05, 'wd': 1e-3},
        {'epochs': 200, 'lr': .01, 'wd': 1e-3},
        {'epochs': 200, 'lr': .05, 'wd': 1e-2},
        {'epochs': 200, 'lr': .01, 'wd': 1e-2},
        {'epochs': 200, 'lr': .05, 'wd': 5e-2},
        {'epochs': 200, 'lr': .01, 'wd': 5e-2},
        
        {'epochs': 500, 'lr': .05, 'wd': 5e-4},
        {'epochs': 500, 'lr': .01, 'wd': 5e-4},
        {'epochs': 500, 'lr': .05, 'wd': 1e-2},
        {'epochs': 500, 'lr': .01, 'wd': 1e-2},
        ]

best_accs = np.zeros((len(EXPS), N_RUNS))
t_i = time.time()
for i in range(N_RUNS):
    for j, exp in enumerate(EXPS):
        arch = GFGCN(IN_DIM, HID_DIM, OUT_DIM, N_LAYERS, K, act=ACT, last_act=LAST_ACT,
                     dropout=DROPOUT, init_h0=h0)
        if NORM:
            S = torch.Tensor(normalize_gso(A, 'both')).to(device)
        else:
            S = torch.Tensor(A).to(device)

        model = GF_NodeClassModel(arch, S, K, masks, LOSS_FN, device=device)
        _, acc = model.train(feat, labels, exp['epochs'], exp['lr'], exp['wd'])

        best_accs[j,i] = np.max(acc["test"])

        print(f'-{i+1}: {exp["epochs"]}-{exp["lr"]}-{exp["wd"]}: {best_accs[j,i]:.3f}')

ellapsed_t = (time.time()-t_i)/60
print(f'----- {ellapsed_t:.2f} mins -----')

# Get table with results
mean_accs = best_accs.mean(axis=1)
std_accs = best_accs.std(axis=1)
index_name = [f'{exp["epochs"]}-{exp["lr"]}-{exp["wd"]}' for exp in EXPS]
table1 = DataFrame(np.vstack((mean_accs, std_accs)).T, columns=['mean accs', 'std'], index=index_name)

-1: 200-0.1-0.0005: 0.622
-1: 200-0.05-0.0005: 0.324
-1: 200-0.01-0.0005: 0.486
-1: 200-0.005-0.0005: 0.270
-1: 200-0.05-0.001: 0.514
-1: 200-0.01-0.001: 0.459
-1: 200-0.05-0.01: 0.865
-1: 200-0.01-0.01: 0.784
-1: 200-0.05-0.05: 0.865
-1: 200-0.01-0.05: 0.730
-1: 500-0.05-0.0005: 0.838
-1: 500-0.01-0.0005: 0.730
-1: 500-0.05-0.01: 0.838
-1: 500-0.01-0.01: 0.838
-2: 200-0.1-0.0005: 0.595
-2: 200-0.05-0.0005: 0.757
-2: 200-0.01-0.0005: 0.514
-2: 200-0.005-0.0005: 0.216
-2: 200-0.05-0.001: 0.514
-2: 200-0.01-0.001: 0.486
-2: 200-0.05-0.01: 0.838
-2: 200-0.01-0.01: 0.784
-2: 200-0.05-0.05: 0.514
-2: 200-0.01-0.05: 0.730
-2: 500-0.05-0.0005: 0.595
-2: 500-0.01-0.0005: 0.486
-2: 500-0.05-0.01: 0.865
-2: 500-0.01-0.01: 0.838
-3: 200-0.1-0.0005: 0.622
-3: 200-0.05-0.0005: 0.514
-3: 200-0.01-0.0005: 0.432
-3: 200-0.005-0.0005: 0.459
-3: 200-0.05-0.001: 0.757
-3: 200-0.01-0.001: 0.486
-3: 200-0.05-0.01: 0.865
-3: 200-0.01-0.01: 0.703
-3: 200-0.05-0.05: 0.514
-3: 200-0.01-0.05: 0.703
-3: 500-0.05

In [5]:
table1

,mean accs,std
200-0.1-0.0005,0.543784,0.090875
200-0.05-0.0005,0.636757,0.142621
200-0.01-0.0005,0.489730,0.079561
200-0.005-0.0005,0.405405,0.076825
200-0.05-0.001,0.708108,0.106474
200-0.01-0.001,0.486486,0.115681
200-0.05-0.01,0.859459,0.013240
200-0.01-0.01,0.767568,0.051280
200-0.05-0.05,0.673514,0.177132
200-0.01-0.05,0.720000,0.020051


## Model params

In [7]:
# h0, norm (norm, not norm, norm H), sep vs joint training, optimizers iters (W vs h)
EXPS = [{'h0': .001, 'epochs': 200, 'epochs_h': 1, 'epochs_W': 1, 'alt': True},
        {'h0': .01, 'epochs': 200, 'epochs_h': 1, 'epochs_W': 1, 'alt': True},
        {'h0': .1, 'epochs': 200, 'epochs_h': 1, 'epochs_W': 1, 'alt': True},
        {'h0': 1, 'epochs': 200, 'epochs_h': 1, 'epochs_W': 1, 'alt': True},
        
        {'h0': .01, 'epochs': 200, 'epochs_h': 1, 'epochs_W': 1, 'alt': False},
        {'h0': 1, 'epochs': 200, 'epochs_h': 1, 'epochs_W': 1, 'alt': False},

        {'h0': 1, 'epochs': 200, 'epochs_h': 10, 'epochs_W': 1, 'alt': True},
        {'h0': 1, 'epochs': 200, 'epochs_h': 1, 'epochs_W': 10, 'alt': True},
        {'h0': .01, 'epochs': 200, 'epochs_h': 10, 'epochs_W': 10, 'alt': True},
        {'h0': 1, 'epochs': 200, 'epochs_h': 10, 'epochs_W': 10, 'alt': True},
        {'h0': 1, 'epochs': 200, 'epochs_h': 25, 'epochs_W': 25, 'alt': True},
        {'h0': 1, 'epochs': 50, 'epochs_h': 50, 'epochs_W': 50, 'alt': True},]


best_accs2 = np.zeros((len(EXPS), N_RUNS))
t_i = time.time()
for i in range(N_RUNS):
    for j, exp in enumerate(EXPS):
        arch = GFGCN(IN_DIM, HID_DIM, OUT_DIM, N_LAYERS, K, act=ACT, last_act=LAST_ACT,
                     dropout=DROPOUT, init_h0=exp['h0'])
        if NORM:
            S = torch.Tensor(normalize_gso(A, 'both')).to(device)
        else:
            S = torch.Tensor(A).to(device)

        if not exp['alt']:
            model = NodeClassModel(arch, S, masks, LOSS_FN, device=device)
            _, acc = model.train(feat, labels, exp['epochs'], LR, WD)
        else:
            model = GF_NodeClassModel(arch, S,  K, masks, LOSS_FN, device=device)
            _, acc = model.train(feat, labels, exp['epochs'], LR, WD, epochs_h=exp['epochs_h'],
                                 epochs_W=exp['epochs_W'])

        best_accs2[j,i] = np.max(acc["test"])

        print(f'-{i+1}: {exp["h0"]}-{exp["epochs_h"]}-{exp["epochs_W"]}-{exp["alt"]}: {best_accs2[j,i]:.3f}')

ellapsed_t = (time.time()-t_i)/60
print(f'----- {ellapsed_t:.2f} mins -----')

# Get table with results
mean_accs = best_accs2.mean(axis=1)
std_accs = best_accs2.std(axis=1)
index_name = [f'{exp["epochs"]}-{exp["h0"]}-{exp["epochs_h"]}-{exp["epochs_W"]}-{exp["alt"]}' for exp in EXPS]
table2 = DataFrame(np.vstack((mean_accs, std_accs)).T, columns=['mean accs', 'std'], index=index_name)

-1: 0.001-1-1-True: 0.865
-1: 0.01-1-1-True: 0.865
-1: 0.1-1-1-True: 0.838
-1: 1-1-1-True: 0.892
-1: 0.01-1-1-False: 0.892
-1: 1-1-1-False: 0.838
-1: 1-10-1-True: 0.892
-1: 1-1-10-True: 0.892
-1: 0.01-10-10-True: 0.865
-1: 1-10-10-True: 0.865
-1: 1-25-25-True: 0.838
-1: 1-50-50-True: 0.838
-2: 0.001-1-1-True: 0.865
-2: 0.01-1-1-True: 0.865
-2: 0.1-1-1-True: 0.892
-2: 1-1-1-True: 0.865
-2: 0.01-1-1-False: 0.865
-2: 1-1-1-False: 0.919
-2: 1-10-1-True: 0.892
-2: 1-1-10-True: 0.865
-2: 0.01-10-10-True: 0.865
-2: 1-10-10-True: 0.919
-2: 1-25-25-True: 0.865
-2: 1-50-50-True: 0.838
-3: 0.001-1-1-True: 0.865
-3: 0.01-1-1-True: 0.865
-3: 0.1-1-1-True: 0.838
-3: 1-1-1-True: 0.838
-3: 0.01-1-1-False: 0.865
-3: 1-1-1-False: 0.892
-3: 1-10-1-True: 0.865
-3: 1-1-10-True: 0.865
-3: 0.01-10-10-True: 0.838
-3: 1-10-10-True: 0.865
-3: 1-25-25-True: 0.865
-3: 1-50-50-True: 0.838
-4: 0.001-1-1-True: 0.892
-4: 0.01-1-1-True: 0.865
-4: 0.1-1-1-True: 0.865
-4: 1-1-1-True: 0.865
-4: 0.01-1-1-False: 0.892
-4: 

In [8]:
table2

,mean accs,std
200-0.001-1-1-True,0.867027,0.041110
200-0.01-1-1-True,0.855135,0.060618
200-0.1-1-1-True,0.869189,0.021185
200-1-1-1-True,0.856216,0.022574
200-0.01-1-1-False,0.860541,0.031221
200-1-1-1-False,0.871351,0.017565
200-1-10-1-True,0.871351,0.019157
200-1-1-10-True,0.860541,0.016537
200-0.01-10-10-True,0.869189,0.016537
200-1-10-10-True,0.869189,0.019757


## Architecture design

In [8]:
# layers, filter order, weightd
EXPS = [{'L': 2, 'K': 2, 'hid_dim': 16},
        {'L': 2, 'K': 3, 'hid_dim': 16},
        {'L': 2, 'K': 4, 'hid_dim': 16},

        {'L': 3, 'K': 2, 'hid_dim': 16},
        {'L': 4, 'K': 2, 'hid_dim': 16},
        {'L': 3, 'K': 3, 'hid_dim': 16},
        {'L': 4, 'K': 3, 'hid_dim': 16},

        {'L': 2, 'K': 2, 'hid_dim': 8},
        {'L': 2, 'K': 2, 'hid_dim': 32},
        {'L': 2, 'K': 2, 'hid_dim': 50},

        {'L': 2, 'K': 3, 'hid_dim': 8},
        {'L': 2, 'K': 3, 'hid_dim': 32},
        {'L': 2, 'K': 3, 'hid_dim': 50},

        {'L': 3, 'K': 2, 'hid_dim': 8},
        {'L': 3, 'K': 2, 'hid_dim': 32},
        {'L': 3, 'K': 2, 'hid_dim': 50},

        {'L': 3, 'K': 3, 'hid_dim': 8},
        {'L': 3, 'K': 3, 'hid_dim': 32},
        {'L': 3, 'K': 3, 'hid_dim': 50},
        ]

best_accs = np.zeros((len(EXPS), N_RUNS))
t_i = time.time()
for i in range(N_RUNS):
    for j, exp in enumerate(EXPS):
        arch = GFGCN(IN_DIM, exp['hid_dim'], OUT_DIM, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                     dropout=DROPOUT, init_h0=h0)
        if NORM:
            S = torch.Tensor(normalize_gso(A, 'both')).to(device)
        else:
            S = torch.Tensor(A).to(device)

        model = GF_NodeClassModel(arch, S,  exp['K'], masks, LOSS_FN, device=device)
        _, acc = model.train(feat, labels, N_EPOCHS, LR, WD)

        best_accs[j,i] = np.max(acc["test"])

        print(f'-{i+1}: {exp["L"]}-{exp["K"]}-{exp["hid_dim"]}: {best_accs[j,i]:.3f}')

ellapsed_t = (time.time()-t_i)/60
print(f'----- {ellapsed_t:.2f} mins -----')

# Get table with results
mean_accs = best_accs.mean(axis=1)
std_accs = best_accs.std(axis=1)
index_name = [f'{exp["L"]}-{exp["K"]}-{exp["hid_dim"]}' for exp in EXPS]
table3 = DataFrame(np.vstack((mean_accs, std_accs)).T, columns=['mean accs', 'std'], index=index_name)

-1: 2-2-16: 0.865
-1: 2-3-16: 0.865
-1: 2-4-16: 0.865
-1: 3-2-16: 0.865
-1: 4-2-16: 0.811
-1: 3-3-16: 0.811
-1: 4-3-16: 0.865
-1: 2-2-8: 0.838
-1: 2-2-32: 0.838
-1: 2-2-50: 0.865
-1: 2-3-8: 0.865
-1: 2-3-32: 0.865
-1: 2-3-50: 0.865
-1: 3-2-8: 0.865
-1: 3-2-32: 0.838
-1: 3-2-50: 0.811
-1: 3-3-8: 0.811
-1: 3-3-32: 0.865
-1: 3-3-50: 0.838
-2: 2-2-16: 0.865
-2: 2-3-16: 0.838
-2: 2-4-16: 0.838
-2: 3-2-16: 0.865
-2: 4-2-16: 0.757
-2: 3-3-16: 0.865
-2: 4-3-16: 0.865
-2: 2-2-8: 0.811
-2: 2-2-32: 0.865
-2: 2-2-50: 0.838
-2: 2-3-8: 0.838
-2: 2-3-32: 0.892
-2: 2-3-50: 0.838
-2: 3-2-8: 0.838
-2: 3-2-32: 0.865
-2: 3-2-50: 0.865
-2: 3-3-8: 0.838
-2: 3-3-32: 0.892
-2: 3-3-50: 0.514
-3: 2-2-16: 0.838
-3: 2-3-16: 0.865
-3: 2-4-16: 0.838
-3: 3-2-16: 0.865
-3: 4-2-16: 0.784
-3: 3-3-16: 0.865
-3: 4-3-16: 0.784
-3: 2-2-8: 0.811
-3: 2-2-32: 0.865
-3: 2-2-50: 0.865
-3: 2-3-8: 0.865
-3: 2-3-32: 0.865
-3: 2-3-50: 0.865
-3: 3-2-8: 0.838
-3: 3-2-32: 0.838
-3: 3-2-50: 0.892
-3: 3-3-8: 0.784
-3: 3-3-32: 0.865
-3: 

In [9]:
table3

,mean accs,std
2-2-16,0.858378,0.019157
2-3-16,0.859459,0.021622
2-4-16,0.848649,0.026481
3-2-16,0.851892,0.018911
4-2-16,0.787027,0.093099
3-3-16,0.838919,0.066852
4-3-16,0.805405,0.092998
2-2-8,0.835676,0.021513
2-2-32,0.862703,0.015135
2-2-50,0.869189,0.018219


## Nonlinearities and loss function

In [10]:
# layers, filter order, weightd
EXPS = [{'act': nn.ReLU(), 'lact': nn.Softmax(dim=1), 'loss': nn.CrossEntropyLoss()},
        {'act': nn.ReLU(), 'lact': nn.Softmax(dim=1), 'loss': nn.NLLLoss()},

        {'act': nn.ReLU(), 'lact': nn.LogSoftmax(dim=1), 'loss': nn.CrossEntropyLoss()},
        {'act': nn.ReLU(), 'lact': nn.LogSoftmax(dim=1), 'loss': nn.NLLLoss()},
        {'act': nn.ReLU(), 'lact': nn.Identity(dim=1), 'loss': nn.CrossEntropyLoss()},
        {'act': nn.ReLU(), 'lact': nn.Identity(dim=1), 'loss': nn.NLLLoss()},

        {'act': nn.ELU(), 'lact': nn.Softmax(dim=1), 'loss': nn.CrossEntropyLoss()},
        {'act': nn.ELU(), 'lact': nn.Softmax(dim=1), 'loss': nn.NLLLoss()},
        {'act': nn.LeakyReLU(), 'lact': nn.Softmax(dim=1), 'loss': nn.CrossEntropyLoss()},
        {'act': nn.LeakyReLU(), 'lact': nn.Softmax(dim=1), 'loss': nn.NLLLoss()},

        {'act': nn.Identity(), 'lact': nn.Softmax(dim=1), 'loss': nn.CrossEntropyLoss()},
        {'act': nn.Identity(), 'lact': nn.Softmax(dim=1), 'loss': nn.NLLLoss()},
        {'act': nn.Identity(), 'lact': nn.Identity(dim=1), 'loss': nn.CrossEntropyLoss()},]

best_accs4 = np.zeros((len(EXPS), N_RUNS))
t_i = time.time()
for i in range(N_RUNS):
    for j, exp in enumerate(EXPS):
        arch = GFGCN(IN_DIM, HID_DIM, OUT_DIM, N_LAYERS, K, act=exp['act'], last_act=exp['lact'],
                     dropout=DROPOUT, init_h0=h0)
        if NORM:
            S = torch.Tensor(normalize_gso(A, 'both')).to(device)
        else:
            S = torch.Tensor(A).to(device)

        model = GF_NodeClassModel(arch, S,  K, masks, exp['loss'], device=device)
        _, acc = model.train(feat, labels, N_EPOCHS, LR, WD)

        best_accs4[j,i] = np.max(acc["test"])

        print(f'-{i+1}: {exp["act"]}-{exp["lact"]}-{exp["loss"]}: {best_accs4[j,i]:.3f}')

ellapsed_t = (time.time()-t_i)/60
print(f'----- {ellapsed_t:.2f} mins -----')

# Get table with results
mean_accs = best_accs4.mean(axis=1)
std_accs = best_accs4.std(axis=1)
index_name = [f'{exp["act"]}-{exp["lact"]}-{exp["loss"]}' for exp in EXPS]
table4 = DataFrame(np.vstack((mean_accs, std_accs)).T, columns=['mean accs', 'std'], index=index_name)

-1: ReLU()-Softmax(dim=1)-CrossEntropyLoss(): 0.865
-1: ReLU()-Softmax(dim=1)-NLLLoss(): 0.865
-1: ReLU()-LogSoftmax(dim=1)-CrossEntropyLoss(): 0.622
-1: ReLU()-LogSoftmax(dim=1)-NLLLoss(): 0.568
-1: ReLU()-Identity()-CrossEntropyLoss(): 0.541
-1: ReLU()-Identity()-NLLLoss(): 0.405
-1: ELU(alpha=1.0)-Softmax(dim=1)-CrossEntropyLoss(): 0.865
-1: ELU(alpha=1.0)-Softmax(dim=1)-NLLLoss(): 0.865
-1: LeakyReLU(negative_slope=0.01)-Softmax(dim=1)-CrossEntropyLoss(): 0.892
-1: LeakyReLU(negative_slope=0.01)-Softmax(dim=1)-NLLLoss(): 0.892
-1: Identity()-Softmax(dim=1)-CrossEntropyLoss(): 0.838
-1: Identity()-Softmax(dim=1)-NLLLoss(): 0.838
-1: Identity()-Identity()-CrossEntropyLoss(): 0.459
-2: ReLU()-Softmax(dim=1)-CrossEntropyLoss(): 0.865
-2: ReLU()-Softmax(dim=1)-NLLLoss(): 0.838
-2: ReLU()-LogSoftmax(dim=1)-CrossEntropyLoss(): 0.568
-2: ReLU()-LogSoftmax(dim=1)-NLLLoss(): 0.676
-2: ReLU()-Identity()-CrossEntropyLoss(): 0.595
-2: ReLU()-Identity()-NLLLoss(): 0.405
-2: ELU(alpha=1.0)-Softma

In [11]:
table4

,mean accs,std
ReLU()-Softmax(dim=1)-CrossEntropyLoss(),0.864865,0.028603
ReLU()-Softmax(dim=1)-NLLLoss(),0.863784,0.019459
ReLU()-LogSoftmax(dim=1)-CrossEntropyLoss(),0.567568,0.057205
ReLU()-LogSoftmax(dim=1)-NLLLoss(),0.588108,0.062235
ReLU()-Identity()-CrossEntropyLoss(),0.602162,0.074712
ReLU()-Identity()-NLLLoss(),0.405405,0.000000
ELU(alpha=1.0)-Softmax(dim=1)-CrossEntropyLoss(),0.872432,0.025946
ELU(alpha=1.0)-Softmax(dim=1)-NLLLoss(),0.863784,0.022261
LeakyReLU(negative_slope=0.01)-Softmax(dim=1)-CrossEntropyLoss(),0.864865,0.022933
LeakyReLU(negative_slope=0.01)-Softmax(dim=1)-NLLLoss(),0.871351,0.020626


# Normalizing the GSO

In [12]:
# # BEST PARAMETERS
# ## Reaining params
# N_RUNS = 25
# N_EPOCHS = 200
# LR = .05
# WD = .01
# DROPOUT = 0

# # BEST PARAMETERS
# ## Architecture params
# N_LAYERS = 2
# K = 2
# HID_DIM = 16

# ## Model params
# h0 = 1
NORM = True

# IN_DIM = feat.shape[1]
# OUT_DIM = n_class


# ACT = nn.ReLU()
# LAST_ACT = nn.Softmax(dim=1)
# LOSS_FN = nn.CrossEntropyLoss()

## Training paramerters

In [13]:
EXPS = [{'epochs': 200, 'lr': .1, 'wd': 5e-4},
        {'epochs': 200, 'lr': .05, 'wd': 5e-4},
        {'epochs': 200, 'lr': .01, 'wd': 5e-4},
        {'epochs': 200, 'lr': .005, 'wd': 5e-4},
        
        {'epochs': 200, 'lr': .05, 'wd': 1e-3},
        {'epochs': 200, 'lr': .01, 'wd': 1e-3},
        {'epochs': 200, 'lr': .05, 'wd': 1e-2},
        {'epochs': 200, 'lr': .01, 'wd': 1e-2},
        {'epochs': 200, 'lr': .05, 'wd': 5e-2},
        {'epochs': 200, 'lr': .01, 'wd': 5e-2},
        
        {'epochs': 500, 'lr': .05, 'wd': 5e-4},
        {'epochs': 500, 'lr': .01, 'wd': 5e-4},
        {'epochs': 500, 'lr': .05, 'wd': 1e-2},
        {'epochs': 500, 'lr': .01, 'wd': 1e-2},]

best_accs = np.zeros((len(EXPS), N_RUNS))
t_i = time.time()
for i in range(N_RUNS):
    for j, exp in enumerate(EXPS):
        arch = GFGCN(IN_DIM, HID_DIM, OUT_DIM, N_LAYERS, K, act=ACT, last_act=LAST_ACT,
                     dropout=DROPOUT, init_h0=h0)
        if NORM:
            S = torch.Tensor(normalize_gso(A, 'both')).to(device)
        else:
            S = torch.Tensor(A).to(device)

        model = GF_NodeClassModel(arch, S, K, masks, LOSS_FN, device=device)
        _, acc = model.train(feat, labels, exp['epochs'], exp['lr'], exp['wd'])

        best_accs[j,i] = np.max(acc["test"])

        print(f'-{i+1}: {exp["epochs"]}-{exp["lr"]}-{exp["wd"]}: {best_accs[j,i]:.3f}')

ellapsed_t = (time.time()-t_i)/60
print(f'----- {ellapsed_t:.2f} mins -----')

# Get table with results
mean_accs = best_accs.mean(axis=1)
std_accs = best_accs.std(axis=1)
index_name = [f'{exp["epochs"]}-{exp["lr"]}-{exp["wd"]}' for exp in EXPS]
table5 = DataFrame(np.vstack((mean_accs, std_accs)).T, columns=['mean accs', 'std'], index=index_name)

/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:43: RuntimeWarning: divide by zero encountered in divide
  D_inv = np.diag(1/S.sum(1))
/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:51: RuntimeWarning: invalid value encountered in matmul
  return D_inv_sqr @ S @ D_inv_sqr


-1: 200-0.1-0.0005: 0.270
-1: 200-0.05-0.0005: 0.270
-1: 200-0.01-0.0005: 0.270
-1: 200-0.005-0.0005: 0.270
-1: 200-0.05-0.001: 0.270
-1: 200-0.01-0.001: 0.270
-1: 200-0.05-0.01: 0.270
-1: 200-0.01-0.01: 0.270
-1: 200-0.05-0.05: 0.270
-1: 200-0.01-0.05: 0.270
-1: 500-0.05-0.0005: 0.270
-1: 500-0.01-0.0005: 0.270
-1: 500-0.05-0.01: 0.270
-1: 500-0.01-0.01: 0.270
-2: 200-0.1-0.0005: 0.270
-2: 200-0.05-0.0005: 0.270
-2: 200-0.01-0.0005: 0.270
-2: 200-0.005-0.0005: 0.270
-2: 200-0.05-0.001: 0.270
-2: 200-0.01-0.001: 0.270
-2: 200-0.05-0.01: 0.270
-2: 200-0.01-0.01: 0.270
-2: 200-0.05-0.05: 0.270
-2: 200-0.01-0.05: 0.270
-2: 500-0.05-0.0005: 0.270
-2: 500-0.01-0.0005: 0.270
-2: 500-0.05-0.01: 0.270
-2: 500-0.01-0.01: 0.270
-3: 200-0.1-0.0005: 0.270
-3: 200-0.05-0.0005: 0.270
-3: 200-0.01-0.0005: 0.270
-3: 200-0.005-0.0005: 0.270
-3: 200-0.05-0.001: 0.270
-3: 200-0.01-0.001: 0.270
-3: 200-0.05-0.01: 0.270
-3: 200-0.01-0.01: 0.270
-3: 200-0.05-0.05: 0.270
-3: 200-0.01-0.05: 0.270
-3: 500-0.05

In [14]:
table5

,mean accs,std
200-0.1-0.0005,0.27027,0.0
200-0.05-0.0005,0.27027,0.0
200-0.01-0.0005,0.27027,0.0
200-0.005-0.0005,0.27027,0.0
200-0.05-0.001,0.27027,0.0
200-0.01-0.001,0.27027,0.0
200-0.05-0.01,0.27027,0.0
200-0.01-0.01,0.27027,0.0
200-0.05-0.05,0.27027,0.0
200-0.01-0.05,0.27027,0.0


## Model params

In [15]:
# h0, norm (norm, not norm, norm H), sep vs joint training, optimizers iters (W vs h)
EXPS = [{'h0': .01, 'epochs': 500, 'epochs_h': 1, 'epochs_W': 1, 'alt': True},
        {'h0': .1, 'epochs': 200, 'epochs_h': 1, 'epochs_W': 1, 'alt': True},
        {'h0': .5, 'epochs': 500, 'epochs_h': 1, 'epochs_W': 1, 'alt': True},
        {'h0': 1, 'epochs': 500, 'epochs_h': 1, 'epochs_W': 1, 'alt': True},
        
        {'h0': 1, 'epochs': 500, 'epochs_h': 1, 'epochs_W': 1, 'alt': False},
        {'h0': 1, 'epochs': 500, 'epochs_h': 10, 'epochs_W': 1, 'alt': True},
        {'h0': 1, 'epochs': 500, 'epochs_h': 1, 'epochs_W': 10, 'alt': True},
        {'h0': 1, 'epochs': 500, 'epochs_h': 10, 'epochs_W': 10, 'alt': True},
        {'h0': 1, 'epochs': 500, 'epochs_h': 25, 'epochs_W': 25, 'alt': True},
        {'h0': 1, 'epochs': 50, 'epochs_h': 50, 'epochs_W': 50, 'alt': True},]


best_accs2 = np.zeros((len(EXPS), N_RUNS))
t_i = time.time()
for i in range(N_RUNS):
    for j, exp in enumerate(EXPS):
        arch = GFGCN(IN_DIM, HID_DIM, OUT_DIM, N_LAYERS, K, act=ACT, last_act=LAST_ACT,
                     dropout=DROPOUT, init_h0=exp['h0'])
        if NORM:
            S = torch.Tensor(normalize_gso(A, 'both')).to(device)
        else:
            S = torch.Tensor(A).to(device)

        if not exp['alt']:
            model = NodeClassModel(arch, S, masks, LOSS_FN, device=device)
            _, acc = model.train(feat, labels, exp['epochs'], LR, WD)
        else:
            model = GF_NodeClassModel(arch, S,  K, masks, LOSS_FN, device=device)
            _, acc = model.train(feat, labels, exp['epochs'], LR, WD, epochs_h=exp['epochs_h'],
                                 epochs_W=exp['epochs_W'])

        best_accs2[j,i] = np.max(acc["test"])

        print(f'-{i+1}: {exp["h0"]}-{exp["epochs_h"]}-{exp["epochs_W"]}-{exp["alt"]}: {best_accs2[j,i]:.3f}')

ellapsed_t = (time.time()-t_i)/60
print(f'----- {ellapsed_t:.2f} mins -----')

# Get table with results
mean_accs = best_accs2.mean(axis=1)
std_accs = best_accs2.std(axis=1)
index_name = [f'{i+1}: {exp["h0"]}-{exp["epochs_h"]}-{exp["epochs_W"]}-{exp["alt"]}' for exp in EXPS]
table6 = DataFrame(np.vstack((mean_accs, std_accs)).T, columns=['mean accs', 'std'], index=index_name)

/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:43: RuntimeWarning: divide by zero encountered in divide
  D_inv = np.diag(1/S.sum(1))
/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:51: RuntimeWarning: invalid value encountered in matmul
  return D_inv_sqr @ S @ D_inv_sqr


-1: 0.01-1-1-True: 0.270
-1: 0.1-1-1-True: 0.270
-1: 1-1-1-True: 0.270
-1: 1-1-1-False: 0.270
-1: 1-10-1-True: 0.270
-1: 1-1-10-True: 0.270
-1: 1-10-10-True: 0.270
-1: 1-25-25-True: 0.270
-1: 1-50-50-True: 0.270
-2: 0.01-1-1-True: 0.270
-2: 0.1-1-1-True: 0.270
-2: 1-1-1-True: 0.270
-2: 1-1-1-False: 0.270
-2: 1-10-1-True: 0.270
-2: 1-1-10-True: 0.270
-2: 1-10-10-True: 0.270
-2: 1-25-25-True: 0.270
-2: 1-50-50-True: 0.270
-3: 0.01-1-1-True: 0.270
-3: 0.1-1-1-True: 0.270
-3: 1-1-1-True: 0.270
-3: 1-1-1-False: 0.270
-3: 1-10-1-True: 0.270
-3: 1-1-10-True: 0.270
-3: 1-10-10-True: 0.270
-3: 1-25-25-True: 0.270
-3: 1-50-50-True: 0.270
-4: 0.01-1-1-True: 0.270
-4: 0.1-1-1-True: 0.270
-4: 1-1-1-True: 0.270
-4: 1-1-1-False: 0.270
-4: 1-10-1-True: 0.270
-4: 1-1-10-True: 0.270
-4: 1-10-10-True: 0.270
-4: 1-25-25-True: 0.270
-4: 1-50-50-True: 0.270
-5: 0.01-1-1-True: 0.270
-5: 0.1-1-1-True: 0.270
-5: 1-1-1-True: 0.270
-5: 1-1-1-False: 0.270
-5: 1-10-1-True: 0.270
-5: 1-1-10-True: 0.270
-5: 1-10-10-

In [16]:
table6

,mean accs,std
25: 0.01-1-1-True,0.27027,0.0
25: 0.1-1-1-True,0.27027,0.0
25: 1-1-1-True,0.27027,0.0
25: 1-1-1-False,0.27027,0.0
25: 1-10-1-True,0.27027,0.0
25: 1-1-10-True,0.27027,0.0
25: 1-10-10-True,0.27027,0.0
25: 1-25-25-True,0.27027,0.0
25: 1-50-50-True,0.27027,0.0


## Architecture design

In [17]:
# layers, filter order, weightd
EXPS = [{'L': 2, 'K': 2, 'hid_dim': 16},
        {'L': 2, 'K': 3, 'hid_dim': 16},
        {'L': 2, 'K': 4, 'hid_dim': 16},

        {'L': 3, 'K': 2, 'hid_dim': 16},
        {'L': 4, 'K': 2, 'hid_dim': 16},
        {'L': 3, 'K': 3, 'hid_dim': 16},
        {'L': 4, 'K': 3, 'hid_dim': 16},

        {'L': 2, 'K': 2, 'hid_dim': 8},
        {'L': 2, 'K': 2, 'hid_dim': 32},
        {'L': 2, 'K': 2, 'hid_dim': 50},

        {'L': 2, 'K': 3, 'hid_dim': 8},
        {'L': 2, 'K': 3, 'hid_dim': 32},
        {'L': 2, 'K': 3, 'hid_dim': 50},

        {'L': 3, 'K': 2, 'hid_dim': 8},
        {'L': 3, 'K': 2, 'hid_dim': 32},
        {'L': 3, 'K': 2, 'hid_dim': 50},

        {'L': 3, 'K': 3, 'hid_dim': 8},
        {'L': 3, 'K': 3, 'hid_dim': 32},
        {'L': 3, 'K': 3, 'hid_dim': 50},
        ]

best_accs = np.zeros((len(EXPS), N_RUNS))
t_i = time.time()
for i in range(N_RUNS):
    for j, exp in enumerate(EXPS):
        arch = GFGCN(IN_DIM, exp['hid_dim'], OUT_DIM, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                     dropout=DROPOUT, init_h0=h0)
        if NORM:
            S = torch.Tensor(normalize_gso(A, 'both')).to(device)
        else:
            S = torch.Tensor(A).to(device)

        model = GF_NodeClassModel(arch, S,  exp['K'], masks, LOSS_FN, device=device)
        _, acc = model.train(feat, labels, N_EPOCHS, LR, WD)

        best_accs[j,i] = np.max(acc["test"])

        print(f'-{i+1}: {exp["L"]}-{exp["K"]}-{exp["hid_dim"]}: {best_accs[j,i]:.3f}')

ellapsed_t = (time.time()-t_i)/60
print(f'----- {ellapsed_t:.2f} mins -----')

# Get table with results
mean_accs = best_accs.mean(axis=1)
std_accs = best_accs.std(axis=1)
index_name = [f'{exp["L"]}-{exp["K"]}-{exp["hid_dim"]}' for exp in EXPS]
table7 = DataFrame(np.vstack((mean_accs, std_accs)).T, columns=['mean accs', 'std'], index=index_name)

/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:43: RuntimeWarning: divide by zero encountered in divide
  D_inv = np.diag(1/S.sum(1))
/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:51: RuntimeWarning: invalid value encountered in matmul
  return D_inv_sqr @ S @ D_inv_sqr


-1: 2-2-16: 0.270
-1: 2-3-16: 0.270
-1: 2-4-16: 0.270
-1: 3-2-16: 0.270
-1: 4-2-16: 0.270
-1: 3-3-16: 0.270
-1: 4-3-16: 0.270
-1: 2-2-8: 0.270
-1: 2-2-32: 0.270
-1: 2-2-50: 0.270
-1: 2-3-8: 0.270
-1: 2-3-32: 0.270
-1: 2-3-50: 0.270
-1: 3-2-8: 0.270
-1: 3-2-32: 0.270
-1: 3-2-50: 0.270
-1: 3-3-8: 0.270
-1: 3-3-32: 0.270
-1: 3-3-50: 0.270
-2: 2-2-16: 0.270
-2: 2-3-16: 0.270
-2: 2-4-16: 0.270
-2: 3-2-16: 0.270
-2: 4-2-16: 0.270
-2: 3-3-16: 0.270
-2: 4-3-16: 0.270
-2: 2-2-8: 0.270
-2: 2-2-32: 0.270
-2: 2-2-50: 0.270
-2: 2-3-8: 0.270
-2: 2-3-32: 0.270
-2: 2-3-50: 0.270
-2: 3-2-8: 0.270
-2: 3-2-32: 0.270
-2: 3-2-50: 0.270
-2: 3-3-8: 0.270
-2: 3-3-32: 0.270
-2: 3-3-50: 0.270
-3: 2-2-16: 0.270
-3: 2-3-16: 0.270
-3: 2-4-16: 0.270
-3: 3-2-16: 0.270
-3: 4-2-16: 0.270
-3: 3-3-16: 0.270
-3: 4-3-16: 0.270
-3: 2-2-8: 0.270
-3: 2-2-32: 0.270
-3: 2-2-50: 0.270
-3: 2-3-8: 0.270
-3: 2-3-32: 0.270
-3: 2-3-50: 0.270
-3: 3-2-8: 0.270
-3: 3-2-32: 0.270
-3: 3-2-50: 0.270
-3: 3-3-8: 0.270
-3: 3-3-32: 0.270
-3: 

In [18]:
table7

,mean accs,std
2-2-16,0.27027,0.0
2-3-16,0.27027,0.0
2-4-16,0.27027,0.0
3-2-16,0.27027,0.0
4-2-16,0.27027,0.0
3-3-16,0.27027,0.0
4-3-16,0.27027,0.0
2-2-8,0.27027,0.0
2-2-32,0.27027,0.0
2-2-50,0.27027,0.0


## Nonlinearities and loss function

In [19]:
# layers, filter order, weightd
EXPS = [{'act': nn.ReLU(), 'lact': nn.Softmax(dim=1), 'loss': nn.CrossEntropyLoss()},
        {'act': nn.ReLU(), 'lact': nn.Softmax(dim=1), 'loss': nn.NLLLoss()},

        {'act': nn.ReLU(), 'lact': nn.LogSoftmax(dim=1), 'loss': nn.CrossEntropyLoss()},
        {'act': nn.ReLU(), 'lact': nn.LogSoftmax(dim=1), 'loss': nn.NLLLoss()},
        {'act': nn.ReLU(), 'lact': nn.Identity(dim=1), 'loss': nn.CrossEntropyLoss()},
        {'act': nn.ReLU(), 'lact': nn.Identity(dim=1), 'loss': nn.NLLLoss()},

        {'act': nn.ELU(), 'lact': nn.Softmax(dim=1), 'loss': nn.CrossEntropyLoss()},
        {'act': nn.ELU(), 'lact': nn.Softmax(dim=1), 'loss': nn.NLLLoss()},
        {'act': nn.LeakyReLU(), 'lact': nn.Softmax(dim=1), 'loss': nn.CrossEntropyLoss()},
        {'act': nn.LeakyReLU(), 'lact': nn.Softmax(dim=1), 'loss': nn.NLLLoss()},

        {'act': nn.Identity(), 'lact': nn.Softmax(dim=1), 'loss': nn.CrossEntropyLoss()},
        {'act': nn.Identity(), 'lact': nn.Softmax(dim=1), 'loss': nn.NLLLoss()},
        {'act': nn.Identity(), 'lact': nn.Identity(dim=1), 'loss': nn.CrossEntropyLoss()},]

best_accs4 = np.zeros((len(EXPS), N_RUNS))
t_i = time.time()
for i in range(N_RUNS):
    for j, exp in enumerate(EXPS):
        arch = GFGCN(IN_DIM, HID_DIM, OUT_DIM, N_LAYERS, K, act=exp['act'], last_act=exp['lact'],
                     dropout=DROPOUT, init_h0=h0)
        if NORM:
            S = torch.Tensor(normalize_gso(A, 'both')).to(device)
        else:
            S = torch.Tensor(A).to(device)

        model = GF_NodeClassModel(arch, S,  K, masks, exp['loss'], device=device)
        _, acc = model.train(feat, labels, N_EPOCHS, LR, WD)

        best_accs4[j,i] = np.max(acc["test"])

        print(f'-{i+1}: {exp["act"]}-{exp["lact"]}-{exp["loss"]}: {best_accs4[j,i]:.3f}')

ellapsed_t = (time.time()-t_i)/60
print(f'----- {ellapsed_t:.2f} mins -----')

# Get table with results
mean_accs = best_accs4.mean(axis=1)
std_accs = best_accs4.std(axis=1)
index_name = [f'{exp["act"]}-{exp["lact"]}-{exp["loss"]}' for exp in EXPS]
table8 = DataFrame(np.vstack((mean_accs, std_accs)).T, columns=['mean accs', 'std'], index=index_name)

/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:43: RuntimeWarning: divide by zero encountered in divide
  D_inv = np.diag(1/S.sum(1))
/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:51: RuntimeWarning: invalid value encountered in matmul
  return D_inv_sqr @ S @ D_inv_sqr


-1: ReLU()-Softmax(dim=1)-CrossEntropyLoss(): 0.270
-1: ReLU()-Softmax(dim=1)-NLLLoss(): 0.270
-1: ReLU()-LogSoftmax(dim=1)-CrossEntropyLoss(): 0.270
-1: ReLU()-LogSoftmax(dim=1)-NLLLoss(): 0.270
-1: ReLU()-Identity()-CrossEntropyLoss(): 0.270
-1: ReLU()-Identity()-NLLLoss(): 0.270
-1: ELU(alpha=1.0)-Softmax(dim=1)-CrossEntropyLoss(): 0.270
-1: ELU(alpha=1.0)-Softmax(dim=1)-NLLLoss(): 0.270
-1: LeakyReLU(negative_slope=0.01)-Softmax(dim=1)-CrossEntropyLoss(): 0.270
-1: LeakyReLU(negative_slope=0.01)-Softmax(dim=1)-NLLLoss(): 0.270
-1: Identity()-Softmax(dim=1)-CrossEntropyLoss(): 0.270
-1: Identity()-Softmax(dim=1)-NLLLoss(): 0.270
-1: Identity()-Identity()-CrossEntropyLoss(): 0.270
-2: ReLU()-Softmax(dim=1)-CrossEntropyLoss(): 0.270
-2: ReLU()-Softmax(dim=1)-NLLLoss(): 0.270
-2: ReLU()-LogSoftmax(dim=1)-CrossEntropyLoss(): 0.270
-2: ReLU()-LogSoftmax(dim=1)-NLLLoss(): 0.270
-2: ReLU()-Identity()-CrossEntropyLoss(): 0.270
-2: ReLU()-Identity()-NLLLoss(): 0.270
-2: ELU(alpha=1.0)-Softma

In [20]:
table8

,mean accs,std
ReLU()-Softmax(dim=1)-CrossEntropyLoss(),0.27027,0.0
ReLU()-Softmax(dim=1)-NLLLoss(),0.27027,0.0
ReLU()-LogSoftmax(dim=1)-CrossEntropyLoss(),0.27027,0.0
ReLU()-LogSoftmax(dim=1)-NLLLoss(),0.27027,0.0
ReLU()-Identity()-CrossEntropyLoss(),0.27027,0.0
ReLU()-Identity()-NLLLoss(),0.27027,0.0
ELU(alpha=1.0)-Softmax(dim=1)-CrossEntropyLoss(),0.27027,0.0
ELU(alpha=1.0)-Softmax(dim=1)-NLLLoss(),0.27027,0.0
LeakyReLU(negative_slope=0.01)-Softmax(dim=1)-CrossEntropyLoss(),0.27027,0.0
LeakyReLU(negative_slope=0.01)-Softmax(dim=1)-NLLLoss(),0.27027,0.0


# Comparing Models

In [29]:
EXPS = [
        {'name': 'Kipf', 'norm': 'none'},
        {'name': 'Kipf', 'norm': 'both'},

        {'name': 'A-GCNN', 'norm': False},
        {'name': 'A-GCNN', 'norm': True},

        {'name': 'H-GCNN', 'norm': False}, # This should be the same as A-GCNN not norm
        {'name': 'H-GCNN', 'norm': True},

        {'name': 'W-GCN-A', 'norm': False},
        {'name': 'W-GCN-A', 'norm': True},
]

In [32]:
best_accs = np.zeros((len(EXPS), N_RUNS))
for i in range(N_RUNS):
    print(f'- RUN: {i+1}')
    for j, exp in enumerate(EXPS):
        # t_i = time.time()
        if exp['name'] == 'Kipf':
            arch = GCNN_2L(IN_DIM, HID_DIM, OUT_DIM, act=ACT, last_act=LAST_ACT,
                           dropout=DROPOUT, norm=exp['norm'])
            S = dgl.from_networkx(nx.from_numpy_array(A)).add_self_loop().to(device)
            
        elif exp['name'] == 'A-GCNN':
            arch = GFGCN(IN_DIM, HID_DIM, OUT_DIM, N_LAYERS, K, act=ACT, last_act=LAST_ACT,
            dropout=DROPOUT, diff_layer=GFGCNLayer, init_h0=h0)
            if exp['norm']:
                S = torch.Tensor(normalize_gso(A, 'both')).to(device)
            else:
                S = torch.Tensor(A).to(device)

        elif exp['name'] == 'H-GCNN':
            arch = GFGCN_Spows(IN_DIM, HID_DIM, OUT_DIM, N_LAYERS, K, act=ACT, last_act=LAST_ACT,
                               dropout=DROPOUT, norm=exp['norm'], dev=device)
            S = torch.Tensor(A).to(device)

        elif exp['name'] == 'W-GCN-A':
            arch = GFGCN(IN_DIM, HID_DIM, OUT_DIM, N_LAYERS, K, act=ACT, last_act=LAST_ACT,
                         dropout=DROPOUT, diff_layer=GFGCN_noh_Layer)
            if exp['norm']:
                S = torch.Tensor(normalize_gso(A, 'both')).to(device)
            else:
                S = torch.Tensor(A).to(device)  
            
        else:
            raise Exception(f'ERROR: Unknown architecture: {exp["name"]}')

        if exp['name'] in ['Kipf', 'W-GCN-A']:
            model = NodeClassModel(arch, S, masks, LOSS_FN, device=device)
        else:
            model = GF_NodeClassModel(arch, S, K, masks, LOSS_FN, device=device)

        loss, acc = model.train(feat, labels, N_EPOCHS, LR, WD)

        best_accs[j,i] = np.max(acc["test"])

        print(f'\t{exp["name"]}-{exp["norm"]}: acc = {best_accs[j,i]:.3f}')


# Print results
mean_accs = best_accs.mean(axis=1)
med_accs = np.median(best_accs, axis=1)
std_accs = best_accs.std(axis=1)

index_name = [f'{exp["name"]}-{exp["norm"]}' for exp in EXPS]
data = np.vstack((mean_accs, med_accs, std_accs)).T
table_comp2 = DataFrame(data, columns=['mean accs', 'med accs', 'std'], index=index_name)
        

- RUN: 1
	Kipf-none: acc = 0.459
	Kipf-both: acc = 0.649
	A-GCNN-False: acc = 0.865


/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:43: RuntimeWarning: divide by zero encountered in divide
  D_inv = np.diag(1/S.sum(1))
/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:51: RuntimeWarning: invalid value encountered in matmul
  return D_inv_sqr @ S @ D_inv_sqr


	A-GCNN-True: acc = 0.270
	H-GCNN-False: acc = 0.838
	H-GCNN-True: acc = 0.892
	W-GCN-A-False: acc = 0.865
	W-GCN-A-True: acc = 0.270
- RUN: 2
	Kipf-none: acc = 0.405
	Kipf-both: acc = 0.649
	A-GCNN-False: acc = 0.865
	A-GCNN-True: acc = 0.270
	H-GCNN-False: acc = 0.892
	H-GCNN-True: acc = 0.838
	W-GCN-A-False: acc = 0.892
	W-GCN-A-True: acc = 0.270
- RUN: 3
	Kipf-none: acc = 0.432
	Kipf-both: acc = 0.649
	A-GCNN-False: acc = 0.865
	A-GCNN-True: acc = 0.270
	H-GCNN-False: acc = 0.865
	H-GCNN-True: acc = 0.838
	W-GCN-A-False: acc = 0.865
	W-GCN-A-True: acc = 0.270
- RUN: 4
	Kipf-none: acc = 0.432
	Kipf-both: acc = 0.622
	A-GCNN-False: acc = 0.865
	A-GCNN-True: acc = 0.270
	H-GCNN-False: acc = 0.865
	H-GCNN-True: acc = 0.865
	W-GCN-A-False: acc = 0.838
	W-GCN-A-True: acc = 0.270
- RUN: 5
	Kipf-none: acc = 0.432
	Kipf-both: acc = 0.649
	A-GCNN-False: acc = 0.892
	A-GCNN-True: acc = 0.270
	H-GCNN-False: acc = 0.865
	H-GCNN-True: acc = 0.838
	W-GCN-A-False: acc = 0.892
	W-GCN-A-True: acc = 

In [ ]:
table_comp2

,mean accs,med accs,std
Kipf-none,0.436757,0.432432,0.036405
Kipf-both,0.628108,0.621622,0.015815
A-GCNN-False,0.855135,0.864865,0.016887
A-GCNN-True,0.270270,0.270270,0.000000
H-GCNN-False,0.861622,0.864865,0.020682
H-GCNN-True,0.796757,0.810811,0.060250
W-GCN-A-False,0.836757,0.837838,0.029089
W-GCN-A-True,0.270270,0.270270,0.000000
